# Sam Jackson, MAE 494, HW3

This code provides answers to the questions posed in Homework 3. The packages used in this homework assignment are numpy, matplotlib.pyplot,scipy.optimize, and pytorch.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import minimize
import torch as t
from torch.autograd import Variable
import math

## Problem 1
### 1.
First, formulating the problem as a least square problem to find $A_{12}$ and $A_{21}$. This is done by simply minimizing the sum of the squared difference between the measured pressures and the pressures calculated from the equilibrium equation for each value of $x_1$. Note:This a constrained problem due to the fact that $x_1$ and $x_2$ are related by being binary.

$$\begin{aligned} &\text{minimize:}_{A_{12},A_{21}} &&\sum^n_{i=1} [p_i - (x_{1i}exp(A_{12}(\frac{A_{21}x_{2i}}{A_{12}x_{1i}+A_{21}x_{2i}})^2)p^{sat}_{water}+x_{2i}exp(A_{21}(\frac{A_{12}x_{1i}}{A_{12}x_{1i}+A_{21}x_{2i}})^2)p^{sat}_{4,1dioxane})]^2 \\ &\text{subject to:} && x_1+x_2=1 \end{aligned}$$

Combining this to be a single unconstrained minimization problem that we can solve...

$$\begin{aligned} &\text{minimize:}_{A_{12},A_{21}} &&\sum^n_{i=1} [p_i - (x_{1i}exp(A_{12}(\frac{A_{21}(1-x_{1i})}{A_{12}x_{1i}+A_{21}(1-x_{1i})})^2)p^{sat}_{water}+(1-x_{1i})exp(A_{21}(\frac{A_{12}x_{1i}}{A_{12}x_{1i}+A_{21}(1-x_{1i})})^2)p^{sat}_{4,1dioxane})]^2 \end{aligned}$$

where $p^{sat}_{water}$ and $p^{sat}_{4,1dioxane}$ are constants that can be determined using given data and the Antoine equation.

To make the problem easier to be implemented in code, the summation function can instead be represented via matrix multiplication, for example if a is a nx1 matrix $a^2=a^Ta$ results in a 1x1 value which is the summation of $a_i^2$ for each value in $a$. Keeping in mind that $x_1$ and $p$ are nx1 matrices (in this case 11x1) as well as being the only matrices in the equation, we can modify the unconstrained problem the summation function into matrix multiplication as such...

$$\begin{aligned} &\text{minimize:}_{A_{12},A_{21}} &&[p - (x_1exp(A_{12}(\frac{A_{21}(1-x_1)}{A_{12}x_1+A_{21}(1-x_1)})^2)p^{sat}_{water}+(1-x_1)exp(A_{21}(\frac{A_{12}x_1}{A_{12}x_1+A_{21}(1-x_1)})^2)p^{sat}_{4,1dioxane})]^T\end{aligned}*[p - (x_1exp(A_{12}(\frac{A_{21}(1-x_1)}{A_{12}x_1+A_{21}(1-x_1)})^2)p^{sat}_{water}+(1-x_1)exp(A_{21}(\frac{A_{12}x_1}{A_{12}x_1+A_{21}(1-x_1)})^2)p^{sat}_{4,1dioxane})]$$

Though this form is less amenable to being written out it is equivalent to the form shown above, and more easily put into code, as such this form will be used when finding the solution shown below.

### 2.

Start with establishing the data given in the problem statement in code. This includes the measured data relating $x_1$ and $p$ and the data to acquire $p^{sat}_{water}$ and $p^{sat}_{1,4 dioxane}$ from the Antoine equation at $T=20^oC$

In [2]:
# Measurement data for x1 and p
x1 = Variable(t.tensor([0.0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1.0]), requires_grad=False)
pm = Variable(t.tensor([28.1,34.4,36.7,36.9,36.8,36.7,36.5,35.4,32.9,27.7,17.5]), requires_grad=False)

# Saturation pressure data
T = 20 #degC
awater=[8.07131, 1730.63, 233.426]
a14dioxane=[7.43155, 1554.679, 240.337]

# Calculating saturation pressures for water and 1,4 dioxane

psatwater= Variable(t.tensor(10**(awater[0]-(awater[1]/(T+awater[2])))), requires_grad=False)
psat14dioxane=Variable(t.tensor(10**(a14dioxane[0]-(a14dioxane[1]/(T+a14dioxane[2])))), requires_grad=False)
print('Saturation pressure of T=20C water =', psatwater.item(), 'mmHg')
print('Saturation pressure of T=20C 1,4 dioxane =', psat14dioxane.item(), 'mmHg')

Saturation pressure of T=20C water = 17.473251342773438 mmHg
Saturation pressure of T=20C 1,4 dioxane = 28.824098587036133 mmHg


Before the minimize problem can be solved using either gradient descent or Newton's method, the gradient of the minimized function with respect to $A_{12}$ and $A_{21}$. This could be done by taking the derivative of the function, as we have done it in previous homeworks, or it could be done using the new method introduced in class by using PyTorch. I have chosen to solve the problem using a gradient descent method with the novel PyTorch way of finding the gradient, as shown below...

In [3]:
# Gradient descent with line search using PyTorch
eps = 1e-1 # Termination criteria
A = Variable(t.tensor([10.0, 5.0]), requires_grad=True) # Defining A's, A[0]=A_12, A[1]=A_21

# Defining loss function as the square sum of the equilibrium relation like defined in 1.
obj = lambda A, x1, psatwater, psat14dioxane:\
        t.dot((pm-(x1*psatwater*t.exp((A[0]*(A[1]*(1-x1))**2/((A[0]*x1+A[1]*(1-x1))**2)))\
        + (1-x1)*psat14dioxane*t.exp((A[1]*((A[0]*x1)**2)/((A[0]*x1+A[1]*(1-x1))**2))))),\
        (pm-(x1*psatwater*t.exp((A[0]*(A[1]*(1-x1))**2/((A[0]*x1+A[1]*(1-x1))**2)))\
        + (1-x1)*psat14dioxane*t.exp((A[1]*((A[0]*x1)**2)/((A[0]*x1+A[1]*(1-x1))**2))))))

# Armijo line search
def line_search(A,x1,psatwater,psatdioxane):
    a = 1.  # initialize step size
    phi = lambda a, A, x1, psatwater, psat14dioxane: obj(A,x1,psatwater,psat14dioxane) - a * 0.8 * A.grad @ A.grad.transpose(0,0)  # define phi as a search criterion
    while phi(a,A,x1,psatwater,psat14dioxane) < obj((A - a * A.grad),x1,psatwater,psat14dioxane):  # if f(x+a*d)>phi(a) then backtrack. d is the search direction
        a = 0.5 * a
    return a

error = 1 # Setting error initially higher than the cutoff


while error >= eps:  # Gradient descent process loops until the gradient is less than the termination criteria
    
    # Defining loss function as the square sum of the equilibrium relation like defined in 1.
    loss = obj(A,x1,psatwater,psat14dioxane)
    loss.backward() # Calculating gradient of loss function w.r.t. A
    error = A.grad.norm() # Setting error equal to the norm of the gradient
    # no_grad() specifies that the operations within this context are not part of the computational graph, i.e., we don't need the gradient descent algorithm itself to be differentiable with respect to A
    with t.no_grad():
        a = line_search(A,x1,psatwater,psat14dioxane) # Perform line search to ensure step size isn't too large
        A -= a*A.grad # Descending the gradient
        
        # need to clear the gradient at every step, or otherwise it will accumulate...
        A.grad.zero_()
        
# Printing out results of gradient descent
print('Optimized Value of A_12 =',A[0].data.numpy())
print('Optimized Value of A_21 =',A[1].data.numpy())
print('Final Value of loss function =',loss.data.numpy())

Optimized Value of A_12 = 1.9578605
Optimized Value of A_21 = 1.6896445
Final Value of loss function = 0.67023647


### 3.
To compare our solution with the data, let's calculate what the model predicts the values of $p$ will be for each value of $x_1$, then subtract the predicted values from the measured values.

In [4]:
# First calculating the predicted values of p for each x1
#for i in range(10):
#    predictionp = (x1[i]*psatwater*t.exp((A[0]*(A[1]**2)*(1-x1[i])*(1-x1[i]))/((A[0]*x1[i]+A[1]*(1-x1[i]))*(A[0]*x1[i]+A[1]*(1-x1[i]))))\
#        + (1-x1[i])*psat14dioxane*t.exp((A[1]*(A[0]**2)*(x1[i]*x1[i]))/((A[0]*x1[i]+A[1]*(1-x1[i]))*(A[0]*x1[i]+A[1]*(1-x1[i])))))
predictionp = (x1*psatwater*t.exp((A[0]*(A[1]*(1-x1))**2/((A[0]*x1+A[1]*(1-x1))**2)))\
        + (1-x1)*psat14dioxane*t.exp((A[1]*((A[0]*x1)**2)/((A[0]*x1+A[1]*(1-x1))**2))))
print('Measured values of p =',pm.data.numpy())
print('Predicted values of p =',predictionp.data.numpy())

#Then subtracting the predicted values of p from the measured values of p to see the difference.
print('Difference in predicted and measured values of p =', (pm-predictionp).data.numpy())

Measured values of p = [28.1 34.4 36.7 36.9 36.8 36.7 36.5 35.4 32.9 27.7 17.5]
Predicted values of p = [28.824099 34.641922 36.451233 36.866104 36.87291  36.748795 36.389854
 35.38533  32.949886 27.733051 17.473251]
Difference in predicted and measured values of p = [-0.7240982  -0.24192047  0.24876785  0.0338974  -0.07291031 -0.04879379
  0.11014557  0.01467133 -0.0498848  -0.03305054  0.02674866]


As we can see, there is a little difference from the predicted and measured values. This could have been guessed from looking at the final value of the loss function in section 2., as it had a fairly small value of ~0.67 while the magnitude of the gradient had reduced to a near zero value, indicating that this was the local minimum. This slight discrepancy in the predicted and measured values likely comes from either error in the used measurements of p or the a values to calculate $p^{sat}_{water}$ and $p^{sat}_{1,4 dioxane}$, or from the fact that a relatively high termination criteria was chosen of $eps=1e^{-1}$. Regardless, the values of $A_{12}$ and $A_{21}$ found above are very close to the best values to fit the measured data, and they fit the data quite well.

## Problem 2

To minimize the following function using Bayesian optimization, 